# Módulo V Práctica 1

## Import

In [24]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/diplo/venv/lib/python3.9/site-packages')

In [2]:
import os

BASE_DIR = '/Users/efraflores/Desktop/EF/Diplo/data/05'
DB = 'sakila'
print(os.listdir(BASE_DIR))

['.DS_Store', 'cred.json']


In [51]:
#!pip install mysqlclient

In [43]:
import json as js
from sqlalchemy import create_engine

with open(os.path.join(BASE_DIR,"cred.json"), "r+") as file:
    cred = js.load(file)
    
conn = create_engine(f"{cred['flavour']}://{cred['username']}:{cred['password']}@{cred['host']}:{cred['port']}/{DB}").connect()

ModuleNotFoundError: No module named 'MySQLdb'

In [7]:
import pandas as pd

dfs = {}
for df in ['film','inventory','rental']:
    dfs[df] = pd.read_sql(con = conn, sql = df)
    print('\n'*2,df.title())
    display(dfs[df].sample())



 Film


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
306,307,FELLOWSHIP AUTUMN,A Lacklusture Reflection of a Dentist And a Hu...,2006,1,None,6,4.99,77,9.99,NC-17,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42




 Inventory


,inventory_id,film_id,store_id,last_update
1016,1017,228,1,2006-02-15 05:09:17




 Rental


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
3284,3287,2005-06-21 06:32:39,4406,62,2005-06-24 09:29:39,2,2006-02-15 21:30:53


## Functions

In [8]:
def multishift(data,id_cols,date_col,shifts,**pivot_args):
    df = data.copy()
    #Make sure the col just have the date (without time)
    df[date_col] = df[date_col].dt.date
    #Merge all column names as a string
    id_col = ','.join(id_cols)
    #And as a column
    df[id_col] = df[id_cols].apply(lambda x:','.join(x.dropna().astype(str)),axis=1)
    #Drop any "id_col"-set that has a lower frequency than the max of the "shifts-list"
    freq = df[id_col].value_counts().to_frame()
    omit_idx = freq[freq[id_col]<=max(shifts)].index.to_list()
    if len(omit_idx)>0:
        df = df[~df[id_col].isin(omit_idx)].copy()
    #Change data structure to build the "shifting"
    df = df.pivot_table(index=[id_col,date_col],
                        **pivot_args,
                        fill_value=0)
    #Concatenate multiple columns if they are
    df.columns = ['_'.join([x for x in col]) if 
                  not isinstance(df.columns[0],str) #First element is not a string
                  else col for col in df.columns]
    #Bring the id_col for taking the set (unique values) in the next loop
    df = df.reset_index()
    #Each shift must be calculated at "id_col" level
    total = pd.DataFrame()
    for row in set(df[id_col]):
        #Set the id_col as index (again) to call all the rows with that id_col
        df_id = df.set_index(id_col).loc[row,:]
        #All possible dates from the min to the max of the subset
        tot_dates = pd.DataFrame(pd.date_range(start=df_id[date_col].min(), 
                                               end=df_id[date_col].max()).date, 
                                 columns=[date_col])
        df_id = df_id.merge(tot_dates,on=date_col,how='right').fillna(0)
        cols = df_id.columns[1:]
        #Start the "shifting"
        aux = df_id.copy()
        for i in shifts:
            aux = aux.join(df_id.iloc[:,1:].shift(i).rename(columns={x:f'{x}_{str(i).zfill(2)}' 
                                                                     for x in cols}))
        aux[id_col] = row
        total = total.append(aux,ignore_index=True)
    return total.set_index(id_cols+[date_col])

## Rental

In [9]:
df = dfs['rental'].iloc[:,:-1].merge(dfs['inventory'].iloc[:,:-1],
                                     on='inventory_id',
                                     how='left')
df.sample()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,film_id,store_id
3694,3697,2005-07-06 10:07:22,1223,182,2005-07-13 14:04:22,2,271,1


In [10]:
df['rental_date'] = pd.to_datetime(df['rental_date'],yearfirst=True)
df['rental_year'] = df['rental_date'].dt.year
df['rental_month'] = df['rental_date'].dt.month
df['rental_dow'] = df['rental_date'].dt.day_name()
df['rental_hrmin'] = df['rental_date'].dt.hour + df['rental_date'].dt.minute/60 + df['rental_date'].dt.second/60**2
df['rental_tw'],qbins = pd.qcut(df['rental_hrmin'],q=6,retbins=True,duplicates='drop')
df['rental_tw'] = df['rental_tw'].astype(str)
df['rental_tw'].value_counts(1)

(12.061, 15.853]    0.166729
(8.073, 12.061]     0.166667
(-0.001, 4.024]     0.166667
(4.024, 8.073]      0.166667
(19.853, 23.999]    0.166667
(15.853, 19.853]    0.166604
Name: rental_tw, dtype: float64

In [11]:
backup = df.copy()
df.sample()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,film_id,store_id,rental_year,rental_month,rental_dow,rental_hrmin,rental_tw
3905,3908,2005-07-06 19:47:26,3980,116,2005-07-13 19:59:26,1,867,1,2005,7,Wednesday,19.790556,"(15.853, 19.853]"


## TAD

In [12]:
df = backup.copy()

In [13]:
df = multishift(df,
           id_cols=['film_id'],
           date_col='rental_date',
           shifts=range(1,8),
           #columns='rental_tw',
           values='customer_id',
           aggfunc='count').dropna().rename(columns={'customer_id':'y'})

df['y'].value_counts(1)

0.0    0.885591
1.0    0.085123
2.0    0.024339
3.0    0.004349
4.0    0.000556
5.0    0.000043
Name: y, dtype: float64

In [14]:
df['y'] = [1 if x>0 else 0 for x in df['y']]
df['y'].value_counts(1)

0    0.885591
1    0.114409
Name: y, dtype: float64

## Model

In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = df[[x for x in df.columns if x not in ['y']]].copy()
y = df['y'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.77,
                                                    random_state=22)
mm_x = MinMaxScaler()

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

model_log = Pipeline(steps=[('scaler', mm_x),
                            ('model', log)])

print('Score: ',model_log.fit(X_train,y_train).score(X_test,y_test))
pd.DataFrame(zip(X_train.columns,model_log[1].coef_[0])).sort_values(1,0,0).reset_index(drop=True)

Score:  0.880847308031774


,0,1
0,customer_id_01,4.824490
1,customer_id_02,3.493519
2,customer_id_03,2.392273
3,customer_id_04,1.527129
4,customer_id_05,0.292719
5,customer_id_06,-1.865553
6,customer_id_07,-11.113391


In [17]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

roc = cross_val_score(estimator=model_log, cv = 4, scoring="roc_auc", X=X_test, y = y_test, n_jobs=-1)
acc = cross_val_score(estimator=model_log, cv = 4, scoring="accuracy", X=X_test, y = y_test, n_jobs=-1)

0.5*(acc.mean()-acc.std()) + 0.5*(roc.mean()-roc.std())

0.823586561645722

## Predict

In [18]:
X.reset_index(drop=True).join(pd.DataFrame(y,columns=['y'])).to_csv(os.path.join(BASE_DIR,'untitled.csv'),index=False)